In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter
from PIL import UnidentifiedImageError
import os

In [ ]:
# Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torch.utils.tensorboard import SummaryWriter
from PIL import UnidentifiedImageError
import os

In [ ]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Custom dataset class to preprocess and filter out corrupted images
class CustomImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except UnidentifiedImageError:
            print(f"Caught UnidentifiedImageError, skipping image at index {index}.")
            return self.__getitem__(index + 1)


In [ ]:
# Load the dataset and automatically split into train and test sets
dataset_path = '/content/drive/MyDrive/dataf/flowers'
dataset = CustomImageFolder(dataset_path, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [ ]:
# Create DataLoaders for train and test setse
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

# Freeze all layers except the last one
for param in model.parameters():
    param.requires_grad = False
model.fc.requires_grad = True

num_classes = len(train_dataset.dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [ ]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
# Initialize TensorBoard writer
writer = SummaryWriter()


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Training loop
num_epochs = 3
train_accuracy_values = []  # List to store training accuracy values
train_loss_values = []      # List to store training loss values

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    train_accuracy_values.append(train_accuracy)  # Store accuracy value
    train_loss_values.append(avg_loss)            # Store loss value

    writer.add_scalar('Loss/train', avg_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {avg_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%")

print('Training finished.')
writer.close()

In [ ]:
# Calculate accuracy on the test set
correct_test = 0
total_test = 0
test_losses = []

model.eval()

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        test_loss = criterion(outputs, labels)
        test_losses.append(test_loss.item())
        _, predicted = torch.max(outputs.data, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

avg_test_loss = sum(test_losses) / len(test_losses)
test_accuracy = 100 * correct_test / total_test

writer = SummaryWriter()
writer.add_scalar('Loss/test', avg_test_loss, num_epochs)
writer.add_scalar('Accuracy/test', test_accuracy, num_epochs)
print(f"Test Loss: {avg_test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")
writer.close()


In [ ]:

# Plot training accuracy and loss
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(range(num_epochs), train_accuracy_values, label='Train Accuracy', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(num_epochs), train_loss_values, label='Train Loss', marker='o')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:

# Plot training and test accuracy
plt.figure(figsize=(8, 5))
plt.plot([0], [train_accuracy_values[0]], 'bo', label='Train Accuracy')
plt.plot([0], [test_accuracy], 'ro', label='Test Accuracy')
plt.xticks([0], ['Epoch 1'])
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Test Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Test phase
test_losses = []

model.eval()

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_losses.append(loss.item())

avg_test_loss = sum(test_losses) / len(test_losses)

# Plot training and test loss
plt.figure(figsize=(8, 5))
plt.plot([0], [train_loss_values[0]], 'bo', label='Train Loss')
plt.plot([0], [avg_test_loss], 'ro', label='Test Loss')
plt.xticks([0], ['Epoch 1'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Test Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Load and test a random image
from PIL import Image
import numpy as np

# Path to a random image in your dataset
random_image_path = '/content/drive/MyDrive/dataf/flowers/daisy test image.jpg'

# Load and preprocess the image
random_image = Image.open(random_image_path)
random_image = transform(random_image).unsqueeze(0).to(device)  # Preprocess and move to GPU

# Get the predicted class
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(random_image)
    _, predicted_class = torch.max(output, 1)

# Map the predicted class index to the class label
class_index = predicted_class.item()
class_label = train_dataset.dataset.classes[class_index]

# Display the result
plt.imshow(np.array(random_image.squeeze().cpu().permute(1, 2, 0)))  # Display the image
plt.title(f"Predicted Class: {class_label}")
plt.axis('off')  # Turn off axis labels
plt.show()